In [ ]:
import requests
from astral.sun import sun
from astral import LocationInfo
from datetime import datetime, timedelta, timezone


# 获取公网IP地址
response = requests.get('https://ipinfo.io')
data = response.json()
# 提取经度和纬度
coordinates = data['loc'].split(',')
latitude = coordinates[0]
longitude = coordinates[1]
# 创建一个 LocationInfo 对象，只提供经纬度信息
location = LocationInfo("", "", "", latitude=latitude, longitude=longitude)
# 获取当前日期和时间，并为其添加时区信息
now = datetime.now(timezone.utc)
def sunrise_sunset(time):
    # 创建一个 Sun 对象
    sun_time = sun(location.observer, date=time)
    # 计算日出和日落时间，以及日落前和日出后的一小时
    sunrise = sun_time['sunrise']
    sunset = sun_time['sunset']
    sunrise_minus_one_hour = sunrise - timedelta(hours=1)
    sunset_plus_one_hour = sunset + timedelta(hours=1)
    return sunrise_minus_one_hour, sunset_plus_one_hour
sunrise_now, sunset_now = sunrise_sunset(now)
if sunrise_now < sunset_now:
    if sunrise_now < now < sunset_now:
        print("现在是白天")
    else:
        print("现在是晚上")
else:
    if sunset_now < now < sunrise_now:
        print("现在是晚上")
    else:
        print("现在是白天")

In [ ]:
import requests
import datetime
import math

# 获取当前日期和时间
now = datetime.datetime.now()
# 获取当前日期是一年中的第几天
day_of_year = now.timetuple().tm_yday
# 获取公网IP地址
response = requests.get('https://ipinfo.io')
data = response.json()
# 提取经度和纬度
coordinates = data['loc'].split(',')
latitude = float(coordinates[0])
longitude = float(coordinates[1])
# 计算太阳的高度角
# 使用简化的算法，不考虑大气折射等因素
solar_declination = -23.44 * math.cos(math.radians(360 * (day_of_year + 10) / 365))
hour_angle = 15 * (now.hour - 12) + (now.minute / 4) + (longitude / 15)
solar_altitude = math.degrees(math.asin(math.sin(math.radians(latitude)) * math.sin(math.radians(solar_declination)) + math.cos(math.radians(latitude)) * math.cos(math.radians(solar_declination)) * math.cos(math.radians(hour_angle))))
# 根据太阳的高度角判断是白天还是晚上
if solar_altitude > 0:
    print("现在是白天")
else:
    print("现在是晚上")


In [ ]:
import qrcode
import math
#网址二维码模块
def qr_code(data):
    # 创建一个QRCode对象
    qr = qrcode.QRCode(version=1,error_correction=qrcode.constants.ERROR_CORRECT_L, box_size=1, border=0,)
    # 设置二维码的数据
    qr.add_data(data)
    # 创建二维码图像
    qr.make(fit=True)
    # 创建一个PIL图像对象
    img = qr.make_image(fill_color="black", back_color="white")
    # 获取图像的宽度和高度
    width, height = img.width, img.height
    height_double = math.ceil(height/2)
    # 转换图像为ASCII字符
    ascii_art = ""
    for y in range(height_double):
        if (y+1)*2-1 >= height:
            for x in range(width):
                if img.getpixel((x, (y+1)*2-2)) == 0:
                    ascii_art += "▀"
                else:
                    ascii_art += " "
        else:
            for x in range(width):
                if img.getpixel((x, (y+1)*2-2)) == 0 and img.getpixel((x, (y+1)*2-1)) == 0:
                    ascii_art += "█"
                elif img.getpixel((x, (y+1)*2-2)) == 0 and img.getpixel((x, (y+1)*2-1)) != 0:
                    ascii_art += "▀"
                elif img.getpixel((x, (y+1)*2-2)) != 0 and img.getpixel((x, (y+1)*2-1)) == 0:
                    ascii_art += "▄"
                else:
                    ascii_art += " "
        ascii_art += "\n"
    # 输出ASCII艺术
    print(ascii_art)
qr_code("https://www.spdb.com.cn")

In [ ]:
import qrcode
import math
def qr_code(data):
    # 创建一个QRCode对象
    qr = qrcode.QRCode(version=1, error_correction=qrcode.constants.ERROR_CORRECT_L, box_size=1, border=0)
    # 设置二维码的数据
    qr.add_data(data)
    # 获取QR Code矩阵
    qr.make(fit=True)
    matrix = qr.make_image(fill_color="black", back_color="white").modules
    # 获取图像的宽度和高度
    width, height = len(matrix), len(matrix)
    height_double = math.ceil(height/2)
    # 转换图像为ASCII字符
    ascii_art = ""
    for y in range(height_double):
        if (y+1)*2-1 >= height:
            for x in range(width):
                if matrix[(y+1)*2-2][x] is True:
                    ascii_art += "▀"
                else:
                    ascii_art += " "
        else:
            for x in range(width):
                if matrix[(y+1)*2-2][x] is True and matrix[(y+1)*2-1][x] is True:
                    ascii_art += "█"
                elif matrix[(y+1)*2-2][x] is True and matrix[(y+1)*2-1][x] is False:
                    ascii_art += "▀"
                elif matrix[(y+1)*2-2][x] is False and matrix[(y+1)*2-1][x] is True:
                    ascii_art += "▄"
                else:
                    ascii_art += " "
        ascii_art += "\n"
    print(ascii_art)





    # 获取QR Code的大小
    length = len(matrix)
    print(length)
    # 黑色和白色的字符
    black_char = "\033[40m  \033[0m"
    white_char = "\033[47m  \033[0m"
    # 遍历QR Code矩阵数据并将其转换为ASCII字符
    ascii_art = ""
    for row in matrix:
        for cell in row:
            if cell:
                ascii_art += black_char
            else:
                ascii_art += white_char
        ascii_art += "\n"
    # 输出ASCII艺术
    print(ascii_art)

qr_code("https://www.spdb.com.cn")

In [ ]:
import yt_dlp
from datetime import datetime, timedelta, timezone
import pandas as pd
# 获取媒体时长和ID模块
def video_format(video_website, video_url, media = "m4a", quality = "480"):
    fail_message = None
    class MyLogger:
        def debug(self, msg):
            pass
        def warning(self, msg):
            pass
        def info(self, msg):
            pass
        def error(self, msg):
            pass
    def duration_and_formats(video_website, video_url):
        fail_message, duration, formats = None, None, None
        try:
            # 初始化 yt_dlp 实例, 并忽略错误
            ydl_opts = {
                'no_warnings': True, 
                'quiet': True,  # 禁止非错误信息的输出
                'logger': MyLogger(),
            }
            ydl = yt_dlp.YoutubeDL(ydl_opts)
            # 使用提供的 URL 提取视频信息
            if info_dict := ydl.extract_info(
                f"{video_website}{video_url}", download = False
            ):
                # 获取视频时长并返回
                duration = info_dict.get('duration')
                formats = info_dict.get('formats')
        except Exception as e:
            fail_message = (str(e)).replace("ERROR: ", "").replace(f"{video_url}: ", "").replace("[youtube] ","")
        return fail_message, duration, formats
    error_reason = {
        "Premieres in ": "\033[31m预播\033[0m|",
        "Premieres ": "\033[31m预播\033[0m|"
    }
    yt_id_count, change_error= 0, None
    fail_message, duration, formats = duration_and_formats(video_website, video_url)
    if fail_message:
        for key in error_reason:
            if key in fail_message:
                change_error = [key, error_reason[key]]
                break
    if change_error:
        fail_message = fail_message.replace(change_error[0], change_error[1])
    else:
        while yt_id_count < 2 and (fail_message is not None or duration is None or formats is None):
            yt_id_count += 1
            #print(f"{datetime.now().strftime('%H:%M:%S')}|{video_url} 无法获取媒体信息\n开始第\033[34m{yt_id_count}\033[0m次重试")
            fail_message, duration, formats = duration_and_formats(video_website, video_url)
    pd.set_option("display.max_rows", None)  # 显示所有行
    pd.set_option("display.max_columns", None)  # 显示所有列
    print(pd.DataFrame(formats))
    if fail_message is None:
        if duration == "" or duration is None:
            return "无法获取时长"
        if formats == "" or formats is None:
            return "无法获取格式"
        duration_and_id = []
        duration_and_id.append(duration)
        # 定义条件判断函数
        def check_resolution(item):
            if "aspect_ratio" in item and (
                isinstance(item["aspect_ratio"], (float, int))
            ):
                if item["aspect_ratio"] >= 1:
                    return item["height"] <= int(quality)
                else:
                    return item["width"] <= int(quality)
            else:
                return False
        def check_ext(item, media):
            return item["ext"] == media if "ext" in item else False
        def check_vcodec(item):
            if "vcodec" in item:
                return "vp" not in item["vcodec"].lower() and "av01" not in item["vcodec"].lower()
            else:
                return False
        #获取最好质量媒体的id
        def best_format_id(formats):
            filesize_max = 0
            format_id_best = ""
            vcodec_best = ""
            for format in formats:
                if (
                    "filesize" in format
                    and (isinstance(format["filesize"], (float, int)))
                    and format["filesize"] > filesize_max
                ):
                    filesize_max = format["filesize"]
                    format_id_best = format["format_id"]
                    vcodec_best = format["vcodec"]
            return format_id_best, vcodec_best
        # 进行筛选
        formats_m4a = list(filter(lambda item: check_ext(item, "m4a") and check_vcodec(item), formats))
        (best_formats_m4a, vcodec_best) = best_format_id(formats_m4a)
        if best_formats_m4a == "" or best_formats_m4a is None:
            return "无法获取音频ID"
        duration_and_id.append(best_formats_m4a)
        if media == "mp4":
            formats_mp4 = list(filter(lambda item: check_resolution(item) and check_ext(item, "mp4") and check_vcodec(item), formats))
            (best_formats_mp4, vcodec_best) = best_format_id(formats_mp4)
            if best_formats_mp4 == ""or best_formats_mp4 is None:
                return "无法获取视频ID"
            duration_and_id.append(best_formats_mp4)
            duration_and_id.append(vcodec_best)
        return duration_and_id
    else:
        return fail_message

In [ ]:
video_format("www.youtube.com/watch?v=", "QQ3CvZzKlhw")

In [ ]:
from bilibili_api import login, user, sync
print("请登录：")
credential = login.login_with_qrcode_term() # 在终端扫描二维码登录
# credential = login.login_with_qrcode() # 使用窗口显示二维码登录
try:
    credential.raise_for_no_bili_jct() # 判断是否成功
    credential.raise_for_no_sessdata() # 判断是否成功
except:
    print("登陆失败。。。")
    exit()
print("欢迎，", sync(user.get_self_info(credential))['name'], "!")

In [ ]:
import threading
import time
from datetime import datetime, timedelta, timezone


# 等待动画模块
def wait_animation(stop_flag):
    animation = "."
    i = 1
    prepare_youtube_print = datetime.now().strftime('%H:%M:%S')
    while not stop_flag[0]:
        print(f"\r{prepare_youtube_print}|YouTube视频 \033[34m下载准备中{animation.ljust(6)}\033[0m", end="")
        if i % 5 == 0:
            animation = "."
        else:
            animation += "."
        i += 1
        time.sleep(0.5)
# 获取视频信息多线程模块
def before_youtube_video_format(stop_flag):
    # 创建线程锁
    youtube_video_format_lock = threading.Lock()
    def youtube_video_format(yt_id):
        ytid_update_format = video_format("https://www.youtube.com/watch?v=", yt_id, youtube_content_ytid_update_format[yt_id]["media"], youtube_content_ytid_update_format[yt_id]["quality"])
        if isinstance(ytid_update_format, list):
            youtube_content_ytid_update_format[yt_id]["format"] = ytid_update_format
        else:
            with youtube_video_format_lock:
                stop_flag[0] = True
                if yt_id_failed ==[]:
                    print("\n")
                yt_id_failed.append(yt_id)
                write_log(f"{channelid_youtube_ids[youtube_content_ytid_update_format[yt_id]['id']]}|{yt_id} {ytid_update_format}")
                del youtube_content_ytid_update_format[yt_id]
    # 创建线程列表
    youtube_content_ytid_update_threads = []
    for yt_id in youtube_content_ytid_update_format.keys():
        thread = threading.Thread(target=youtube_video_format, args=(yt_id,))
        youtube_content_ytid_update_threads.append(thread)
        thread.start()
    # 等待所有线程完成
    for thread in youtube_content_ytid_update_threads:
        thread.join()
    stop_flag[0] = True
    if yt_id_failed ==[]:
        print("\033[34m已完成\033[0m")
# 创建共享的标志变量
stop_flag = [False]  # 使用列表来存储标志变量
# 创建两个线程分别运行等待动画和其他代码，并传递共享的标志变量
prepare_youtube_1 = threading.Thread(target=wait_animation, args=(stop_flag,))
prepare_youtube_2 = threading.Thread(target=before_youtube_video_format, args=(stop_flag,))
# 启动两个线程
prepare_youtube_1.start()
prepare_youtube_2.start()
# 等待两个线程结束
prepare_youtube_1.join()
prepare_youtube_2.join()


In [ ]:
import requests
url = "https://www.youtube.com/feeds/videos.xml?channel_id=UCBR8-601B28hp2BmDPdntcQ"
try:
    response = requests.get(f"{url}", timeout = 5)
    response.raise_for_status()
except Exception as e:
    if response.status_code in [404]:
        print(response.text)
    else:
        print(e)
else:
    print(response.text)

In [2]:
data = {i: i**2 for i in range(100)}
chunk_size = 20

# 字典拆分模块
def split_dict(data, chunk_size=100, firse_item_only=False):
    chunks = []
    if chunk_size == 0:
        return [{}]
    else:
        if firse_item_only:
            end_value = chunk_size
        else:
            end_value = len(data)
        for i in range(0, end_value, chunk_size):
            chunk = dict(list(data.items())[i:i+chunk_size])
            chunks.append(chunk)
        return chunks

split_dict(data, chunk_size)


[{0: 0,
  1: 1,
  2: 4,
  3: 9,
  4: 16,
  5: 25,
  6: 36,
  7: 49,
  8: 64,
  9: 81,
  10: 100,
  11: 121,
  12: 144,
  13: 169,
  14: 196,
  15: 225,
  16: 256,
  17: 289,
  18: 324,
  19: 361},
 {20: 400,
  21: 441,
  22: 484,
  23: 529,
  24: 576,
  25: 625,
  26: 676,
  27: 729,
  28: 784,
  29: 841,
  30: 900,
  31: 961,
  32: 1024,
  33: 1089,
  34: 1156,
  35: 1225,
  36: 1296,
  37: 1369,
  38: 1444,
  39: 1521},
 {40: 1600,
  41: 1681,
  42: 1764,
  43: 1849,
  44: 1936,
  45: 2025,
  46: 2116,
  47: 2209,
  48: 2304,
  49: 2401,
  50: 2500,
  51: 2601,
  52: 2704,
  53: 2809,
  54: 2916,
  55: 3025,
  56: 3136,
  57: 3249,
  58: 3364,
  59: 3481},
 {60: 3600,
  61: 3721,
  62: 3844,
  63: 3969,
  64: 4096,
  65: 4225,
  66: 4356,
  67: 4489,
  68: 4624,
  69: 4761,
  70: 4900,
  71: 5041,
  72: 5184,
  73: 5329,
  74: 5476,
  75: 5625,
  76: 5776,
  77: 5929,
  78: 6084,
  79: 6241},
 {80: 6400,
  81: 6561,
  82: 6724,
  83: 6889,
  84: 7056,
  85: 7225,
  86: 7396,
  87: 75

In [1]:
def number_to_ordinal(number):
    # 检查是否为 10 到 20 之间的数字，这个范围的序数词后缀都是 'th'
    if 10 <= number % 100 <= 20:
        suffix = 'th'
    else:
        # 对于其他数字，根据个位数选择相应的序数词后缀
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(number % 10, 'th')
    
    # 将数字和后缀连接在一起返回
    return str(number) + suffix

# 示例
for i in range(1, 600):
    # 调用函数，将数字转化为英文的排名
    ordinal_rank = number_to_ordinal(i)
    # 打印结果
    print(f"{i} 对应的英文排名为：{ordinal_rank}")


1 对应的英文排名为：1st
2 对应的英文排名为：2nd
3 对应的英文排名为：3rd
4 对应的英文排名为：4th
5 对应的英文排名为：5th
6 对应的英文排名为：6th
7 对应的英文排名为：7th
8 对应的英文排名为：8th
9 对应的英文排名为：9th
10 对应的英文排名为：10th
11 对应的英文排名为：11th
12 对应的英文排名为：12th
13 对应的英文排名为：13th
14 对应的英文排名为：14th
15 对应的英文排名为：15th
16 对应的英文排名为：16th
17 对应的英文排名为：17th
18 对应的英文排名为：18th
19 对应的英文排名为：19th
20 对应的英文排名为：20th
21 对应的英文排名为：21st
22 对应的英文排名为：22nd
23 对应的英文排名为：23rd
24 对应的英文排名为：24th
25 对应的英文排名为：25th
26 对应的英文排名为：26th
27 对应的英文排名为：27th
28 对应的英文排名为：28th
29 对应的英文排名为：29th
30 对应的英文排名为：30th
31 对应的英文排名为：31st
32 对应的英文排名为：32nd
33 对应的英文排名为：33rd
34 对应的英文排名为：34th
35 对应的英文排名为：35th
36 对应的英文排名为：36th
37 对应的英文排名为：37th
38 对应的英文排名为：38th
39 对应的英文排名为：39th
40 对应的英文排名为：40th
41 对应的英文排名为：41st
42 对应的英文排名为：42nd
43 对应的英文排名为：43rd
44 对应的英文排名为：44th
45 对应的英文排名为：45th
46 对应的英文排名为：46th
47 对应的英文排名为：47th
48 对应的英文排名为：48th
49 对应的英文排名为：49th
50 对应的英文排名为：50th
51 对应的英文排名为：51st
52 对应的英文排名为：52nd
53 对应的英文排名为：53rd
54 对应的英文排名为：54th
55 对应的英文排名为：55th
56 对应的英文排名为：56th
57 对应的英文排名为：57th
58 对应的英文排名为：58th
59 对应的英文排名为：59th
60 对应的英文排名为：60t